<a href="https://colab.research.google.com/github/SoltanReza/classification/blob/main/Projet_main_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prérequis

In [ ]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')
import sys
my_local_drive='/content/gdrive/My Drive/Colab Notebooks/ML_FDS'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd


!pip install umap-learn[plot]
!pip install holoviews
!pip install  ipykernel
!pip install joblib
!pip install scikeras
!pip install scikit-learn==1.3.2 scikeras==0.12.0
# !pip install scikit-learn scikeras

# eventuellement ne pas oublier de relancer le kernel du notebook
# !pip install tensorflow==2.17.0

In [ ]:
# Importation des différentes librairies utiles pour le notebook

#Sickit learn met régulièrement à jour des versions et
#indique des futurs warnings.
#ces deux lignes permettent de ne pas les afficher.
import warnings
from tensorflow.keras import mixed_precision
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.metrics import classification_report
from datetime import datetime
# librairies générales
import joblib
import pickle
import pandas as pd
from scipy.stats import randint
import numpy as np
import string
import time
import base64
from keras import Input, Model
from tensorflow.keras.optimizers import Adam, SGD

import re
import sys
import copy
import random
from numpy import mean
from numpy import std
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from scipy.linalg import sqrtm
from tensorflow.keras.models import load_model
from google.colab import auth
import google.auth

# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import plotly.graph_objs as go
import plotly.offline as py
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# TensorFlow et keras
from keras import layers
from keras import models
from keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Conv2D, MaxPooling2D,UpSampling2D, Dropout, Flatten, BatchNormalization
from keras.preprocessing import image
from tqdm import tqdm
from keras.models import load_model
from sklearn.model_selection import KFold
from keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
import os
from os import listdir
from os.path import isfile, join
import cv2
import glob
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
import io
import tempfile
from skimage.feature import hog
from sklearn.svm import SVC
import numpy as np
from tensorflow.keras.applications.inception_v3 import InceptionV3 # Import InceptionV3

from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import KFold, GridSearchCV
from tensorflow.keras.applications import ResNet50


##Utils

In [ ]:

def calculate_fid(real_images, generated_images, num_samples=100, batch_size=32):
    inception_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))

    def preprocess_images(images):
        images = tf.image.resize(images, (299, 299))
        images = tf.clip_by_value(images, -1.0, 1.0)
        images = (images + 1) / 2.0
        images = preprocess_input(images * 255.0)
        return images

    real_images = real_images[:num_samples]
    generated_images = generated_images[:num_samples]

    real_images = tf.convert_to_tensor(real_images, dtype=tf.float32)
    generated_images = tf.convert_to_tensor(generated_images, dtype=tf.float32)

    if real_images.shape[0] == 0 or generated_images.shape[0] == 0:
        raise ValueError("Aucune image valide dans l'échantillon.")

    real_features = inception_model.predict(preprocess_images(real_images), batch_size=batch_size, verbose=0)
    generated_features = inception_model.predict(preprocess_images(generated_images), batch_size=batch_size, verbose=0)

    if real_features.shape[0] == 0 or generated_features.shape[0] == 0:
        raise ValueError("Aucune caractéristique extraite, vérifiez les entrées.")

    mu_real = np.mean(real_features, axis=0)
    mu_gen = np.mean(generated_features, axis=0)
    sigma_real = np.cov(real_features, rowvar=False)
    sigma_gen = np.cov(generated_features, rowvar=False)

    epsilon = 1e-6
    sigma_real += np.eye(sigma_real.shape[0]) * epsilon
    sigma_gen += np.eye(sigma_gen.shape[0]) * epsilon

    diff = mu_real - mu_gen

    covmean = sqrtm(sigma_real.dot(sigma_gen))
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    fid = np.sum(diff**2) + np.trace(sigma_real + sigma_gen - 2 * covmean)
    return float(fid) if np.isfinite(fid) else float('inf')

def evaluate_gan(generator, real_images, num_samples=100, batch_size=32, save_dir=None, epoch=None):
    # Tester avec stddev variable
    noise_stddevs = [0.5, 1.0]
    for stddev in noise_stddevs:
        noise = tf.random.normal([num_samples, 100], mean=0.0, stddev=stddev)
        print(f"Shape du bruit (stddev={stddev}):", noise.shape)

        generated_images = generator(noise, training=False)

        real_images = tf.convert_to_tensor(real_images, dtype=tf.float32)
        if tf.reduce_min(real_images) < -1.5 or tf.reduce_max(real_images) > 1.5:
            print("Attention : real_images ne semble pas normalisé à [-1, 1], normalisation appliquée")
            real_images = tf.clip_by_value(real_images, -1.0, 1.0)

        print(f"Generated images min: {tf.reduce_min(generated_images).numpy():.4f}, max: {tf.reduce_max(generated_images).numpy():.4f}")

        try:
            fid_score = calculate_fid(real_images, generated_images, num_samples=num_samples, batch_size=batch_size)
            print(f"FID Score (stddev={stddev}): {fid_score:.4f}")
        except Exception as e:
            print(f"Erreur lors du calcul du FID (stddev={stddev}): {e}")
            fid_score = None

        generated_images = tf.clip_by_value((generated_images + 1) / 2.0, 0.0, 1.0)

        plt.figure(figsize=(10, 10))
        for i in range(min(25, num_samples)):
            plt.subplot(5, 5, i+1)
            plt.imshow(generated_images[i])
            plt.axis('off')
        title = f"Generated Images)"
        plt.suptitle(title, fontsize=12)

        if save_dir:
            os.makedirs(save_dir, exist_ok=True)
            save_path = os.path.join(save_dir, f'evaluation_epoch_{epoch if epoch else "test"}_stddev_{stddev}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.png')
            plt.savefig(save_path)
            print(f"Images sauvegardées à : {save_path}")

        plt.show()

        # Bruit fixe
        fixed_noise = tf.random.normal([num_samples, 100], mean=0.0, stddev=stddev, seed=42)
        fixed_generated_images = generator(fixed_noise, training=False)
        fixed_generated_images = tf.clip_by_value((fixed_generated_images + 1) / 2.0, 0.0, 1.0)

        plt.figure(figsize=(10, 10))
        for i in range(min(25, num_samples)):
            plt.subplot(5, 5, i+1)
            plt.imshow(fixed_generated_images[i])
            plt.axis('off')
        plt.suptitle(f"Fixed Noise Images (stddev={stddev}, Epoch {epoch if epoch else 'Unknown'})", fontsize=12)

        if save_dir:
            fixed_save_path = os.path.join(save_dir, f'fixed_noise_epoch_{epoch if epoch else "test"}_stddev_{stddev}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.png')
            plt.savefig(fixed_save_path)
            print(f"Images à bruit fixe sauvegardées à : {fixed_save_path}")

        plt.show()


def plot_augmented_images(images, datagen, num_examples=5):
    images = (images + 1) / 2.0
    plt.figure(figsize=(15, 3))
    for i in range(num_examples):
        # Générer une image augmentée
        img = images[i:i+1]  # Batch d'une seule image
        aug_iter = datagen.flow(img, batch_size=1)
        aug_img = next(aug_iter)[0]
        plt.subplot(1, num_examples, i+1)
        plt.imshow(aug_img)
        plt.axis('off')
    plt.show()

def load_images(image_dir, target_size=(64, 64)):
    images = []
    for filename in os.listdir(image_dir):
        img_path = os.path.join(image_dir, filename)
        img = Image.open(img_path).resize(target_size)
        img = np.array(img) / 127.5 - 1.0  # Normalisation à [-1, 1]
        images.append(img)
    return np.array(images)

def visualize_colorization(grayscale_images, rgb_images, predicted_images, num_examples=5, save_dir=None):
    plt.figure(figsize=(15, 5 * num_examples))
    for i in range(num_examples):
        # Image grayscale
        plt.subplot(num_examples, 3, i * 3 + 1)
        plt.imshow((grayscale_images[i] + 1) / 2.0, cmap='gray')
        plt.title('Grayscale')
        plt.axis('off')

        # Image colorisée
        plt.subplot(num_examples, 3, i * 3 + 2)
        plt.imshow((predicted_images[i] + 1) / 2.0)
        plt.title('Colorized')
        plt.axis('off')

        # Image originale
        plt.subplot(num_examples, 3, i * 3 + 3)
        plt.imshow((rgb_images[i] + 1) / 2.0)
        plt.title('Original')
        plt.axis('off')

    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        plt.savefig(os.path.join(save_dir, 'colorization_results.png'))
    plt.show()
    plt.close()

##Charger les données

In [ ]:

tiger_dir = '/content/gdrive/My Drive/Colab Notebooks/ML_FDS/Data_Project/Tiger-Fox-Elephant/fox'
tiger_images = load_images(tiger_dir)
my_classes=['fox','Fox_negative_class']

#tiger_images = tiger_images[0:4]
fox_data = tf.data.Dataset.from_tensor_slices(tiger_images).shuffle(1000).batch(32)
print(f"Nombre d'images chargées : {len(tiger_images)}")


## Data augmentation


In [ ]:


def augment_dataset(x_train, datagen, augmentation_factor=10):
    augmented_images = []
    batch_size = 32
    x_train_normalized = (x_train + 1) / 2.0

    for i in range(len(x_train)):
        img = x_train_normalized[i:i+1]
        aug_iter = datagen.flow(img, batch_size=1)
        for _ in range(augmentation_factor):
            aug_img = next(aug_iter)[0]
            augmented_images.append(aug_img)

    augmented_images = np.array(augmented_images) * 2.0 - 1.0
    x_train_augmented = np.concatenate([x_train, augmented_images], axis=0)

    return x_train_augmented

x_train = load_images(tiger_dir)

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit((x_train + 1) / 2.0)
plot_augmented_images(x_train, datagen)
old_x_train = x_train
x_train = augment_dataset(old_x_train, datagen, augmentation_factor=9)

print(f"Taille dataset original : {old_x_train.shape}")
print(f"Taille dataset augmenté : {x_train.shape}")
#sauvegarder le dataset augmenté pour l'utiliser dans le DCGAN
np.save('/content/gdrive/My Drive/Colab Notebooks/ML_FDS/tiger_images_augmented.npy', x_train)

#Autoencodeurs

In [ ]:
IMAGE_SIZE=64
def load_and_preprocess_images(folder_path, img_size=(64, 64), test_size=0.2):
    images = []

    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):  # Filtrer les formats d'image
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, img_size)
                # Normaliser les pixels entre -1 et 1
                img = img.astype('float32') / 255
                images.append(img)

    # Convertir la liste en tableau NumPy
    X = np.array(images)

    # # Reshape pour ajouter la dimension de canal (comme dans l'exemple Fashion MNIST)
    # X = np.reshape(X, (X.shape[0], img_size[0], img_size[1], 3))  # 3 pour RGB

    # Diviser en ensembles d'entraînement et de test
    X_train, X_test = train_test_split(X, test_size=test_size, random_state=42)

    return X_train, X_test


folder_path = "Data_Project/Tiger-Fox-Elephant/fox"  # Remplace par le chemin réel

X_train, X_test = load_and_preprocess_images(folder_path, img_size=(64, 64), test_size=0.2)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

print(f"X_train min: {X_train.min()}, max: {X_train.max()}")
print(f"X_test min: {X_test.min()}, max: {X_test.max()}")



def plot_images(images, title, num_images=5):
    plt.figure(figsize=(15, 4), dpi=100)
    plt.suptitle(title, fontsize=20)
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(images[i], interpolation='nearest')  # Display the image
        plt.axis('off')  # Hide axes
    plt.tight_layout()
    plt.show()

# Plot X_train images
plot_images(X_train, "X_train Images")

# Plot X_test images
plot_images(X_test, "X_test Images")


In [ ]:
# Taille de l'image d'entrée
input_shape = (64, 64, 3)
# Initialisation du modèle
model = Sequential()
# Couche d'entrée
model.add(Input(shape=input_shape, name="Input"))
# Encodeur
model.add(Conv2D(32, (3, 3), activation='relu', padding='same',name="Conv2D_1"))
model.add(MaxPooling2D((2, 2), padding='same', name="MaxPooling_1"))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name="Conv2D_2"))
model.add(MaxPooling2D((2, 2), padding='same', name="MaxPooling_2"))
model.add(Conv2D(4, (3, 3), activation='relu', padding='same', name="Conv2D_Bottleneck"))
# Décodeur
model.add(Conv2D(4, (3, 3), activation='relu', padding='same',name="Conv2D_3"))
model.add(UpSampling2D((2, 2), name="UpSampling_1"))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same',name="Conv2D_4"))
model.add(UpSampling2D((2, 2), name="UpSampling_2"))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same',name="Conv2D_5"))

# Couche de sortie modifiée pour produire une image 64x64 avec 3 canaux
model.add(Conv2D(3, (3, 3), activation='tanh', padding='same', name="Output"))

# Compilation du modèle
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='mean_squared_error')
# Affichage de la structure du modèle
model.summary()

In [ ]:
epochs = 800
batch_size = 32
model_filename = 'AutoEncoderMnist.keras'
# Callback pour sauvegarder le modèle toutes les époques
checkpoint_callback = ModelCheckpoint(
"autoencoder_fashionmnist_epoch{epoch:02d}.keras", # Nom du fichier
save_freq='epoch',
save_weights_only=False,
verbose=1
)
# Entraînement du modèle
history = model.fit(
X_train, X_train,
epochs=epochs,
batch_size=batch_size,
validation_data=(X_test, X_test),
verbose=1,
callbacks=[checkpoint_callback]
)
model.save(model_filename)
print(f'Modèle sauvegardé sous le nom : {model_filename}')

In [ ]:
model_filename = 'AutoEncoderMnist.keras'
model = load_model(model_filename)

# Prédiction avec le modèle
pred = model.predict(X_test)

# Affichage des images d'entrée
fig, axes = plt.subplots(1, 5, figsize=(15, 4))
fig.suptitle("Images Entrée", fontsize=20)
for i in range(5):
    axes[i].imshow(X_test[i], interpolation='nearest')
    axes[i].axis('off')
plt.tight_layout()
plt.show()

print(len(pred))
# Affichage des images de sortie (prédictions)
fig, axes = plt.subplots(1, 5, figsize=(15, 4))
fig.suptitle("Images Sortie", fontsize=20)
for i in range(5):
    axes[i].imshow(pred[i] , cmap='gray' ,interpolation='nearest')
    axes[i].axis('off')
plt.tight_layout()
plt.show()

###Autoencodeurs bruité

In [ ]:
# Définition du facteur de bruit
noise_factor = 0.3
# Ajout du bruit aux images d'entraînement et de test
X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0,scale=1.0, size=X_train.shape)
X_test_noisy = X_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_test.shape)
# Limitation des valeurs des images bruitées entre 0 et 1
X_train_noisy = np.clip(X_train_noisy, 0., 1.)
X_test_noisy = np.clip(X_test_noisy, 0., 1.)
# Affichage des images bruitées
plt.figure(figsize=(20, 2))
for i in range(5): # Afficher 5 images
  ax = plt.subplot(1, 5, i + 1)
  plt.imshow(X_test_noisy[i], cmap='gray')
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
# Taille de l'image d'entrée
input_shape = (64, 64, 3)

# Initialisation du modèle
model = keras.Sequential(name="denoising_autoencoder")

# Encodeur
model.add(layers.Input(shape=input_shape, name="Input"))
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', name="Conv2D_1"))
model.add(layers.MaxPooling2D((2, 2), padding='same', name="MaxPooling_1"))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', name="Conv2D_2"))
model.add(layers.MaxPooling2D((2, 2), padding='same', name="MaxPooling_2"))
model.add(layers.Conv2D(4, (3, 3), activation='relu', padding='same', name="Conv2D_Bottleneck"))  # Bottleneck

# Décodeur
model.add(layers.Conv2D(4, (3, 3), activation='relu', padding='same', name="Conv2D_3"))
model.add(layers.UpSampling2D((2, 2), name="UpSampling_1"))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', name="Conv2D_4"))
model.add(layers.UpSampling2D((2, 2), name="UpSampling_2"))
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', name="Conv2D_5"))

# Couche de sortie (pour la reconstruction de l'image)
model.add(layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same', name="Output"))  # Sigmoid for [0, 1] output

# Compilation du modèle
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='mse')  # MSE loss for image reconstruction

# Affichage de la structure du modèle
model.summary()

In [ ]:
epochs = 30
batch_size = 32
model_filename = 'AutoEncoderNoise.keras'
# Callback pour sauvegarder le modèle toutes les époques
checkpoint_callback = ModelCheckpoint(
"autoencoder_noise_fashionmnist_epoch{epoch:02d}.keras",
save_freq='epoch',
save_weights_only=False,
verbose=1
)
# Entraînement du modèle
history = model.fit(
X_train_noisy, X_train,
epochs=epochs,
batch_size=batch_size,
validation_data=(X_test_noisy, X_test),
verbose=1,
callbacks=[checkpoint_callback]
)
#Sauvegarde du modèle
model.save(model_filename)
print(f'Modèle sauvegardé sous le nom : {model_filename}')

In [ ]:
model_filename = 'AutoEncoderNoise.keras'
model = load_model(model_filename)

# Prédiction sur les images bruitées
pred = model.predict(X_test_noisy)
# Affichage des images bruitées
plt.figure(figsize=(20, 4))
plt.suptitle("Images bruitées", fontsize=20)
for i in range(5):
  ax = plt.subplot(2, 5, i + 1)
  plt.imshow(X_test_noisy[i], cmap='gray')
  ax.axis('off')
plt.tight_layout()
plt.show()
# Affichage des images de sortie après débruitage
plt.figure(figsize=(20, 4))
plt.suptitle("Images débruitées", fontsize=20)
for i in range(5):
  ax = plt.subplot(2, 5, i + 1)
  plt.imshow(pred[i], cmap='gray')
  ax.axis('off')
plt.tight_layout()
plt.show()

#Modèle

In [ ]:
#Les variables globales
epochs=600
taille_backup=100 # le nombre d'epochs après les quelles on fait backup des images et on les stocke
IMAGE_SIZE = 32

batch_size = 32


In [ ]:
nameFile = f'generator_epoch_'

def build_generator():
    model = models.Sequential()
    model.add(layers.Dense(4*4*256, input_dim=100))
    model.add(layers.Reshape((4, 4, 256)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, 4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, 4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, 4, strides=4, padding='same', activation='tanh'))
    return model

def build_discriminator():
    model = models.Sequential()
    model.add(layers.Conv2D(64, 4, strides=2, padding='same', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Conv2D(128, 4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Conv2D(256, 4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

generator = build_generator()
discriminator = build_discriminator()

cross_entropy = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

@tf.function
def train_step(images):
    noise = tf.random.normal([tf.shape(images)[0], 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

def generate_and_save_images(model, epoch, test_input, save_dir):
    predictions = model(test_input, training=False)
    predictions = (predictions + 1) / 2.0

    fig = plt.figure(figsize=(10, 10))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i])
        plt.axis('off')

    plt.savefig(os.path.join(save_dir, f'image_at_epoch_{epoch:04d}.png'))
    plt.close()

def train_gan(dataset, epochs, save_dir):
    global nameFile  # Déclarer nameFile comme globale
    os.makedirs(save_dir, exist_ok=True)
    test_noise = tf.random.normal([16, 100])

    for epoch in range(epochs):
        start = time.time()
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        if (epoch + 1) % 3 == 0:
            nameFile = f'generator_epoch_{epoch+1}_{datetime.now()}.h5'
            print(nameFile)
            generate_and_save_images(generator, epoch + 1, test_noise, save_dir)
            keras.saving.save_model(generator,os.path.join(save_dir, nameFile))


        print(f"Epoch {epoch+1}, Time: {time.time() - start:.2f}s, Gen Loss: {gen_loss:.4f}, Disc Loss: {disc_loss:.4f}")
#dataset augmented
# dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(1000).batch(batch_size)
save_dir = '/content/gdrive/My Drive/Colab Notebooks/ML_FDS/gan_outputs'
train_gan(dataset, epochs=epochs, save_dir=save_dir)


In [ ]:

save_dir = '/content/gdrive/My Drive/Colab Notebooks/ML_FDS/gan_outputs'
generator = load_model(os.path.join(save_dir, nameFile))

generator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
generator.compile(loss='binary_crossentropy', optimizer=generator_optimizer)

tiger_images = np.load('/content/gdrive/My Drive/Colab Notebooks/ML_FDS/tiger_images_augmented.npy')
tiger_images = tf.convert_to_tensor(tiger_images, dtype=tf.float32)
evaluate_gan(generator, tiger_images, num_samples=100, batch_size=32, save_dir=save_dir, epoch=300)

In [ ]:
nameFile = "generator_epoch_300_20250506_220726.h5"

#Colorisation

In [ ]:
epochs_colorization=40
batch_size_colorization= 32
def rgb_to_grayscale(images):
    images = (images + 1) / 2.0
    grayscale = tf.image.rgb_to_grayscale(images)
    grayscale = grayscale * 2.0 - 1.0
    return grayscale.numpy()

def prepare_colorization_dataset(image_path):
    rgb_images = np.load(image_path)
    grayscale_images = rgb_to_grayscale(rgb_images)
    dataset = tf.data.Dataset.from_tensor_slices((grayscale_images, rgb_images))
    dataset = dataset.shuffle(1000).batch(32)
    return dataset, grayscale_images, rgb_images

def build_colorization_model():
    inputs = tf.keras.Input(shape=(64, 64, 1))

    # Encodeur
    e1 = layers.Conv2D(64, 4, strides=2, padding='same', activation='relu')(inputs)
    e2 = layers.Conv2D(128, 4, strides=2, padding='same', activation='relu')(e1)
    e3 = layers.Conv2D(256, 4, strides=2, padding='same', activation='relu')(e2)
    e4 = layers.Conv2D(512, 4, strides=2, padding='same', activation='relu')(e3)
    # Décodeur
    d1 = layers.Conv2DTranspose(256, 4, strides=2, padding='same', activation='relu')(e4)
    d1 = layers.Concatenate()([d1, e3])  # Skip connection
    d2 = layers.Conv2DTranspose(128, 4, strides=2, padding='same', activation='relu')(d1)
    d2 = layers.Concatenate()([d2, e2])
    d3 = layers.Conv2DTranspose(64, 4, strides=2, padding='same', activation='relu')(d2)
    d3 = layers.Concatenate()([d3, e1])

    outputs = layers.Conv2DTranspose(3, 4, strides=2, padding='same', activation='tanh')(d3)
    model = models.Model(inputs, outputs)
    return model


def train_and_evaluate_colorization(image_path, epochs=50, batch_size=32):
    dataset, grayscale_images, rgb_images = prepare_colorization_dataset(image_path)

    model = build_colorization_model()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='mse',
        metrics=['mae']
    )

    save_dir = '/content/gdrive/My Drive/Colab Notebooks/ML_FDS/colorization_models'
    file_name=f'generator_epoch_{datetime.now()}'

    os.makedirs(save_dir, exist_ok=True)
    checkpoint = ModelCheckpoint(
        os.path.join(save_dir, file_name),
        monitor='val_loss',
        save_best_only=True,
        mode='min'
    )
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    history = model.fit(
        dataset,
        epochs=epochs,
        callbacks=[checkpoint, early_stopping]
    )

    num_examples = 5
    test_grayscale = grayscale_images[:num_examples]
    predicted_rgb = model.predict(test_grayscale)

    vis_save_dir = '/content/gdrive/My Drive/Colab Notebooks/ML_FDS/colorization_results'
    visualize_colorization(test_grayscale, rgb_images[:num_examples], predicted_rgb,
                         num_examples=num_examples, save_dir=vis_save_dir)

    return model, history

image_path = '/content/gdrive/My Drive/Colab Notebooks/ML_FDS/tiger_images_augmented.npy'

colorization_model, training_history = train_and_evaluate_colorization(image_path, epochs=epochs_colorization, batch_size=batch_size_colorization)